Build chemical descriptors/molecular representation (e.g. ECFP, CoulombMatrix, MolecularGraphs etc) as input into deep learning network

In [1]:
import numpy as np
import pandas as pd
import deepchem as dc
import os

from rdkit.Chem.Fingerprints import FingerprintMols

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras import regularizers

/Users/AikRui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Import compound data

In [2]:
data = pd.read_pickle(os.path.join(os.getcwd(), 'data.pkl'))
print (data.shape)
data.head(1)

(28263, 19)


,smiles,category,mw_freebase,alogp,hba,hbd,psa,rtb,acd_logp,acd_logd,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,mol,agrochemical
0,Cl.O=C(NCc1ccncc1)[C@@H]2CCCN2C(=O)[C@@H]3CCCN3,toxin,302.38,0.44,4,2,74.33,4,1.04,-0.77,338.84,1,22,0.85,302.174,6,2,<rdkit.Chem.rdchem.Mol object at 0x12bc10fb8>,0


Add Fingerprint descriptor using RDKIT

In [3]:
def fingerprint(mol):
    return np.ndarray.flatten(np.array(FingerprintMols.FingerprintMol(mol)))

In [4]:
data['fingerprint'] = data['mol'].apply(fingerprint)

Assign feature (X) and agrochemical classifier (Y)

In [5]:
X = data['fingerprint']
Y = data['agrochemical']

Pad molecular fingerprint with zeros to make them all into same size for conversion into numpy array later

__QUESTION: CAN WE DO PADDING ON MOLECULAR FINGERPRINT? DOES IT CHANGE THE STRUCTURE?"__

In [6]:
def molecular_fingerprint_padding(x):
    result = np.zeros((2048,))
    result[:x.shape[0]] = x
    return result
X = X.apply(molecular_fingerprint_padding)

Stack X into one numpy array

In [7]:
X = np.stack(X)
X.shape

(28263, 2048)

Convert Y into numpy array

In [8]:
Y = np.ndarray.flatten(np.array(Y))
Y.shape

(28263,)

Standard scaling

In [9]:
st = StandardScaler()
X = st.fit_transform(X)

Split into training set and testing set

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Define metrics for classification

In [14]:
def classification_metrics(y_true, y_pred):
    if (y_true.shape != y_pred.shape):
        print ("Shapes of two arrays do not match.")
        return 
    tp, fp, tn, fn = 0, 0, 0, 0
    total = y_true.shape[0]
    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            tp += 1
        elif true == 0 and pred == 0:
            tn += 1
        elif true == 1 and pred == 0:
            fn += 1
        else:
            fp += 1
    print ("True positive:", float(tp)/total)
    print ("True negative:", float(tn)/total)
    print ("False positive:", float(fp)/total)
    print ("False negative:", float(fn)/total)
    print ()
    print ("Accuracy: ", float(tn + tp)/total)

Build models (*Maybe we can try compare using dense neural network and random forest?*)

1) Simple sequential neural network

In [11]:
def build_simple_model():
    model = Sequential()
    model.add(Dense(128, input_dim=2048, activation='relu', kernel_regularizer=regularizers.l2(0.)))
    model.add(Dense(8, input_dim=128, activation='softmax'))
    model.add(Dense(1, input_dim=8, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = build_simple_model()

In [12]:
history = model.fit(x_train, y_train, nb_epoch=25, batch_size=32)

/Users/AikRui/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/25
22610/22610 [==============================] - 6s 253us/step - loss: 0.4969 - acc: 0.8115
Epoch 2/25
22610/22610 [==============================] - 6s 248us/step - loss: 0.3774 - acc: 0.8715
Epoch 3/25
22610/22610 [==============================] - 5s 219us/step - loss: 0.3251 - acc: 0.8868
Epoch 4/25
22610/22610 [==============================] - 5s 223us/step - loss: 0.2920 - acc: 0.8903
Epoch 5/25
22610/22610 [==============================] - 5s 236us/step - loss: 0.2737 - acc: 0.9035
Epoch 6/25
22610/22610 [==============================] - 6s 254us/step - loss: 0.2567 - acc: 0.9057
Epoch 7/25
22610/22610 [==============================] - 6s 253us/step - loss: 0.2337 - acc: 0.9108
Epoch 8/25
22610/22610 [==============================] - 5s 230us/step - loss: 0.2232 - acc: 0.9161
Epoch 9/25
22610/22610 [==============================] - 5s 239us/step - loss: 0.2311 - acc: 0.9144
Epoch 10/25
22610/22610 [==============================] - 6s 255us/step - loss: 0.2080 - a

In [13]:
y_pred = model.predict(x_test)
y_pred = np.around(np.ndarray.flatten(y_pred))

In [15]:
classification_metrics(y_test, y_pred)

True positive: 0.45568724571024233
True negative: 0.44595789846099415
False positive: 0.07182027242172298
False negative: 0.02653458340704051

Accuracy:  0.9016451441712365


2) Use random forest model

In [16]:
rfc = RandomForestClassifier(n_estimators=200, max_depth=2, random_state=0, n_jobs=-1)

In [17]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
y_pred = rfc.predict(x_test)

In [19]:
classification_metrics(y_test, y_pred)

True positive: 0.25809304793914734
True negative: 0.3962497788784716
False positive: 0.12152839200424553
False negative: 0.2241287811781355

Accuracy:  0.654342826817619


3) Convolutional Neural Network